### Predict on 1 image to make sure the network is tied together correctly

In [ ]:
#1. Load the trained network weights
#2. Run the net forward and get an array of predictions
#3. Compare the result to the label

using Mocha, Images, ImageView, Colors, HDF5, MLBase
backend = CPUBackend()
init(backend)

image_x = 32
image_y = 32
image_depth = 3

test_index = 6;

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
    data=Array[zeros(Float32, image_x, image_y, image_depth, 1)])

#Copy over the rest of the layers from the training network
conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip1])

#Build the network
common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the network snapshot from the training data
load_snapshot(run_net, "snapshots/snapshot-070000.jld")

#Now run one test through the network to ensure it is set up correctly
fid = h5open("cifar_data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

get_layer(run_net, "data").data[1][:,:,image_depth,1] = features[:,:,image_depth,test_index]

offset = 1  #my tags vector is 1-indexed but the labels vector is 
            #is 0-indexed.  So I have to add an offset of 1 to the label value to 
            # match my predictive value 
println("\nCorrect label index: ", Int64(labels[test_index])+offset)

forward(run_net)
println()
println("Label prob vector:")
println(run_net.output_blobs[:prob].data)

# Use MLBase to classify the output blob

prediction = classify(run_net.output_blobs[:prob].data)
println("Prediction is: ", prediction[1])



### Predict on 1 with visualization

In [ ]:
#1. Load the trained network weights
#2. Run the net forward and get an array of predictions
#3. Use MLBase to classify the prediction
#4. Load the tags and GT labels
#5. Use the tags and GT to annotate the image

using Mocha, Images, ImageView, Colors, HDF5, MLBase
backend = CPUBackend()
init(backend)

image_x = 32
image_y = 32
image_depth = 3

test_index = 12;

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
    data=Array[zeros(Float32, image_x, image_y, image_depth, 1)])

#Copy over the rest of the layers from the training network
conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip1])

#Build the network
common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the network snapshot from the training data
load_snapshot(run_net, "snapshots/snapshot-070000.jld")

#Now run one test through the network to ensure it is set up correctly
fid = h5open("cifar_data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

get_layer(run_net, "data").data[1][:,:,image_depth,1] = features[:,:,image_depth,test_index]

#read the meta_labels file
tags = []
fname = "batches.meta.txt"
open(fname) do file
    for line in eachline(file)
        line = strip(line)
        if length(line)>0 
            push!(tags, strip(line))
        end
    end
end

offset = 1  #my tags vector is 1-indexed but the labels vector is 
            #is 0-indexed.  So I have to add an offset of 1 to the label value to 
            # match my predictive value 
gt = Int64(labels[test_index])+offset
println("Ground truth label: ", gt, "-",tags[gt])


# Run the net forward and use MLBase to classify the output blob
forward(run_net)
prediction = classify(run_net.output_blobs[:prob].data)
println("Prediction is:      ", prediction[1], "-", tags[prediction[1]])

# Load and annotate the image
#read the image and label data files
vis_fid = h5open("cifar_data/visual.hdf5", "r")
vis_data = read(vis_fid, "data")
image = test_index

#create the image
img = convert(Image, vis_data[:,:,:,image])
img["spatialorder"]=["x","y"]

#create the string label
l1 = string("GT: ", tags[gt])
l2 = string("Pred: ", tags[prediction[1]])

#set up the label location on the 32x32 image
x1 = 4
y1 = 30
x2 = 4
y2 = 3

#annotate the image
imgc, imgslice = ImageView.view(img);
idx = ImageView.annotate!(imgc, imgslice, ImageView.AnnotationText(x1, y1, l1, color=RGB(1,1,1), fontsize=2, halign="left"))
idx = ImageView.annotate!(imgc, imgslice, ImageView.AnnotationText(x2, y2, l2, color=RGB(1,1,1), fontsize=2, halign="left"))
Tk.set_size(ImageView.toplevel(imgc), 300, 300)



### Predict on 100 and display on canvas

In [12]:
using Mocha, Images, ImageView, Colors, HDF5, MLBase

#1. Load the trained network weights
#2. Set up a loop to run the net forward and get an array of predictions
#3.    Use MLBase to classify the prediction
#4.    Load the tags and GT labels
#5.    Use the tags and GT to annotate the image
#6.    Add the image to a canvas

using Mocha, Images, ImageView, Colors, HDF5, MLBase
backend = CPUBackend()
init(backend)

image_x = 32
image_y = 32
image_depth = 3

test_index = 12;

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
    data=Array[zeros(Float32, image_x, image_y, image_depth, 1)])

#Copy over the rest of the layers from the training network
conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip1])

#Build the network
common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the network snapshot from the training data
load_snapshot(run_net, "snapshots/snapshot-070000.jld")

#Now run one test through the network to ensure it is set up correctly
fid = h5open("data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

#Load the displayable images
vis_fid = h5open("data/visual.hdf5", "r")
vis_data = read(vis_fid, "data")

#Set up some constants used in the loop
#set up the label location on the 32x32 image
x1 = 4
y1 = 30
x2 = 4
y2 = 3
offset = 1  #my tags vector is 1-indexed but the labels vector is 
                  #is 0-indexed.  So I have to add an offset of 1 to the label value to 
                  # match my predictive value 
println("Loading Tags...")
tags = []
fname = "batches.meta.txt"
open(fname) do file
    for line in eachline(file)
        line = strip(line)
        if length(line)>0 
            push!(tags, strip(line))
        end
    end
end

#Set canvas size
N = 10
#Create a canvas to paint on
c = canvasgrid(N,N)
font = round(N/2) # set fontsize relative to N

#set up a list to capture results
gts = []
preds = []

println("Running net forward for an $N x $N canvas...")
printed_flag = 0;
for j in 1:N for k in 1:N
    if j%10==0 && printed_flag != j
        println("     Running column $j ...") 
        printed_flag = j
    end
    #convert the j,k vals into a straight index
    i = (j-1)*10 + k
    #Load the ith image and label 
    get_layer(run_net, "data").data[1][:,:,image_depth,1] = features[:,:,image_depth,i]
    gt = Int64(labels[i])+offset
    push!(gts, gt)

    # Run the net forward and use MLBase to classify the output blob
    forward(run_net)
    prediction = classify(run_net.output_blobs[:prob].data)
    push!(preds, prediction[1])
        
    #create the visual image for the ith sample
    img = convert(Image, vis_data[:,:,:,i])
    img["spatialorder"]=["x","y"]

    #create the string label
    l1 = string("GT: ", tags[gt])
    l2 = string("Pred: ", tags[prediction[1]])

    #annotate the image
    c2, slice = ImageView.view(c[j,k], img)
    idx = ImageView.annotate!(c2, slice, ImageView.AnnotationText(x1, y1, l1, color=RGB(1,0,0), fontsize=font, halign="left"))
    idx = ImageView.annotate!(c2, slice, ImageView.AnnotationText(x2, y2, l2, color=RGB(1,0,0), fontsize=font, halign="left"))
  end
end
#    Tk.set_size(ImageView.toplevel(imgc), 300, 300)

gts = convert(Array{Int64}, gts)
preds = convert(Array{Int64}, preds)
confusmat(10, gts, preds)

20-Oct 14:24:59:INFO:root:Constructing net imagenet on Mocha.CPUBackend...
20-Oct 14:24:59:INFO:root:Topological sorting 11 layers...
20-Oct 14:24:59:INFO:root:Setup layers...
20-Oct 14:24:59:INFO:root:Network constructed!
20-Oct 14:24:59:INFO:root:Loading existing model from snapshots/snapshot-070000.jld
20-Oct 14:24:59:DEBUG:root:Loading parameters for layer conv1
20-Oct 14:24:59:DEBUG:root:Loading parameters for layer conv2
20-Oct 14:24:59:DEBUG:root:Loading parameters for layer conv3
20-Oct 14:24:59:DEBUG:root:Loading parameters for layer ip1
Loading Tags...
Running net forward for an 10 x 10 canvas...
     Running column 10 ...


10x10 Array{Int64,2}:
  8  0  2  0  0  0  0  0  0  0
  9  3  2  0  0  0  0  0  0  0
  3  0  5  0  0  0  0  0  0  0
  6  0  3  2  0  0  0  0  0  0
  6  0  3  0  0  0  0  0  0  0
  1  0  3  0  0  0  0  0  0  0
  6  1  5  0  0  0  0  0  0  0
  3  0  3  0  0  0  0  0  0  1
  9  0  1  0  0  0  0  0  1  0
 11  1  1  0  0  0  0  0  0  1

### Build confusion matrix for all tests

In [17]:
using Mocha, Images, ImageView, Colors, HDF5, MLBase

#1. Load the trained network weights
#2. Set up a loop to run the net forward and get an array of predictions
#3.    Use MLBase to classify the prediction
#4.    Load the tags and GT labels
#5.    Use the tags and GT to annotate the image
#6.    Add the image to a canvas

backend = CPUBackend()
init(backend)

image_x = 32
image_y = 32
image_depth = 3

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
    data=Array[zeros(Float32, image_x, image_y, image_depth, 1)])

#Copy over the rest of the layers from the training network
conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip1])

#Build the network
common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the network snapshot from the training data
load_snapshot(run_net, "snapshots/snapshot-070000.jld")

#Now run one test through the network to ensure it is set up correctly
fid = h5open("data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

#Load the displayable images
vis_fid = h5open("data/visual.hdf5", "r")
vis_data = read(vis_fid, "data")

#Set up some constants used in the loop
#set up the label location on the 32x32 image
N = 1000 # number of samples
x1 = 4
y1 = 30
x2 = 4
y2 = 3
offset = 1  #my tags vector is 1-indexed but the labels vector is 
                  #is 0-indexed.  So I have to add an offset of 1 to the label value to 
                  # match my predictive value 
println("Loading Tags...")
tags = []
fname = "batches.meta.txt"
open(fname) do file
    for line in eachline(file)
        line = strip(line)
        if length(line)>0 
            push!(tags, strip(line))
        end
    end
end

#set up a list to capture results
gts = []
preds = []

printed_flag = 0;
for j in 1:N
    if j%100==0
        println("     Running iteration $j ...") 
    end
 
    #Load the jth image and label 
    get_layer(run_net, "data").data[1][:,:,image_depth,1] = features[:,:,image_depth,j]
    gt = Int64(labels[j])+offset
    push!(gts, gt)

    # Run the net forward and use MLBase to classify the output blob
    forward(run_net)
    prediction = classify(run_net.output_blobs[:prob].data)
    push!(preds, prediction[1])
        
end
#    Tk.set_size(ImageView.toplevel(imgc), 300, 300)

gts = convert(Array{Int64}, gts)
preds = convert(Array{Int64}, preds)
println(correctrate(gts, preds))
confusmat(10, gts, preds)

20-Oct 16:22:28:INFO:root:Constructing net imagenet on Mocha.CPUBackend...
20-Oct 16:22:28:INFO:root:Topological sorting 11 layers...
20-Oct 16:22:28:INFO:root:Setup layers...
20-Oct 16:22:28:INFO:root:Network constructed!
20-Oct 16:22:28:INFO:root:Loading existing model from snapshots/snapshot-070000.jld
20-Oct 16:22:28:DEBUG:root:Loading parameters for layer conv1
20-Oct 16:22:28:DEBUG:root:Loading parameters for layer conv2
20-Oct 16:22:28:DEBUG:root:Loading parameters for layer conv3
20-Oct 16:22:28:DEBUG:root:Loading parameters for layer ip1
Loading Tags...


LoadError: LoadError: SystemError: opening file batches.meta.txt: No such file or directory
while loading In[17], in expression starting on line 80